In [ ]:
#import packages
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import pandas as pd
import numpy as np

## OAuth 2.0 Setup

In [ ]:
##Select scope at https://developers.google.com/calendar/auth. Scopes determine access to calendars
#scopes = ['https://www.googleapis.com/auth/calendar']

In [ ]:
##Authorize access to calendar
#flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)
#credentials = flow.run_console()

In [ ]:
##Save credentials so you only have to authorize once 
#pickle.dump(credentials, open("token.pkl", "wb"))

In [ ]:
#Import credentials
credentials = pickle.load(open("token.pkl", "rb"))

In [ ]:
#
service = build("calendar", "v3", credentials=credentials)

## Get Calendars

In [ ]:
#Query Calendars
result = service.calendarList().list().execute()
result

## Get Calendar Events

In [ ]:
#Query events
events = service.events().list(calendarId="johnvanzalk@gmail.com",timeMin="2020-06-10T00:00:00Z").execute()

In [ ]:
#Get all items (events)
events = events.get('items', [])
events

In [ ]:
#Create empty lists to store event info
emails = []
statuses = []
event_names = []
dates = []

#Loop through events. If event has certain title, loop though attendees and append values to lists
for event in events:
    try:
        if event['summary'] == 'Birthday Party':
            try:
                for attendee in event['attendees']:
                        
                        emails.append(attendee['email'])
                        statuses.append(attendee['responseStatus'])
                        event_names.append(event['summary'])
                        dates.append(event['start']['dateTime'])
                        
            except: 
                pass
    except:
        pass

In [ ]:
#Combine lists in to dataframe
attendees = pd.DataFrame(
    {'email': emails,
     'status': statuses,
     'event': event_names,
     'date': dates
    })

attendees.head()

In [ ]:
#Filter attendees that did not accept meeting invite
attendees = attendees[attendees['status']=='accepted']

In [ ]:
#Convert datetime to just date
attendees['date'] = pd.to_datetime(attendees['date'])
attendees['date'] = attendees['date'].dt.date
attendees.head()